File to process Kaggle Dataset into usuable form for this project: https://www.kaggle.com/datasets/arashnic/building-sites-power-consumption-dataset

In [ ]:
import pandas as pd

energy_df = pd.read_csv('buildings_data\power-laws-forecasting-energy-consumption-training-data.csv', sep=';')
outdoor_temp_df = pd.read_csv('buildings_data\power-laws-forecasting-energy-consumption-weather.csv', sep=';')
base_temp_df = pd.read_csv('buildings_data\power-laws-forecasting-energy-consumption-metadata.csv', sep=';')

In [ ]:
# Currently not in use because one-to-one mapping exists!
from datetime import datetime

# Find the closest temperature given the current timestamp and the weather measurements
# of a specific site
MAX_DIFFERENCE = 12*60*60 # difference for half a day
def findClosestTemp(time, weather_measurements):
    curr_time = datetime.fromisoformat(time)
    # initialize closest_time to the first timestamp
    closest_time = datetime.fromisoformat(weather_measurements.iloc[0]['Timestamp'])
    for index, row in weather_measurements.iterrows():
        record_time = datetime.fromisoformat(row['Timestamp'])

        # difference between current row and given time
        record_diff = abs((record_time - curr_time).total_seconds()) 
        # difference between closest row and given time
        closest_diff = abs((closest_time - curr_time).total_seconds())

        # if the difference in the current row is less than the closest diff
        # AKA if the timestamp in the current row is closer than all the other timestamps we have seen
        if(record_diff < closest_diff):
            closest_time = record_time

    diff = abs((closest_time-curr_time).total_seconds())
    if(diff > MAX_DIFFERENCE):
        return "" # return empty string if closest temp is not within half a day
    return closest_time

In [ ]:
# keep track of rows to be added to new dataframe
timestamps = []
base_temps = []
outdoor_temps = []
energy_consumptions = []
stop = False
MAX_SIZE = 10000

# concept: 
# go through energy_df by siteid 
# retrieve the weather measurements for that siteid from outdoor_temp_df (might be unsucessful if weather data doesn't exist for this site)
# if step 2 successful
     # add timestamp (energy_df), basetemp (base_temp_df), outdoor_temp (outdoor_temp_df), and energy_consumption (energy_df) to appropriate lists

df_list = energy_df.groupby(energy_df['SiteId'])
for item in df_list:
    if(stop):
        break
    
    e_df = item[1]
    site_id = e_df.iloc[0]['SiteId']
    bt = base_temp_df[base_temp_df['SiteId'] == site_id].iloc[0]['BaseTemperature']
    weather_measurements = outdoor_temp_df[(outdoor_temp_df['SiteId'] == site_id)]
    if(len(weather_measurements) > 0):
        for index,row in e_df.iterrows():
            if(stop):
                break 

            time = row['Timestamp']
            energy = row['Value']
            corresponding_time = weather_measurements[weather_measurements['Timestamp'] == time]
            if(len(corresponding_time) == 1): # we find a mapped measurement
                ot = corresponding_time.iloc[0]["Temperature"] # retreive outdoor temp

                # add data to arrays
                timestamps.append(time)
                base_temps.append(bt)
                outdoor_temps.append(ot)
                energy_consumptions.append(energy)

                # cap our data
                if(len(timestamps) > MAX_SIZE):
                    stop = True
                # print("base temp:", str(bt), "outdoor temp:", str(ot), "energy:", str(energy))
            
# create new dataframe
data = {'Timestamp':timestamps, 'IndoorTemperature':base_temps, 'OutdoorTemperature':outdoor_temps, 'EnergyConsumption':energy_consumptions}
df = pd.DataFrame(data)

In [41]:
print(outdoor_temp_df.head())

                   Timestamp  Temperature   Distance  SiteId
0  2017-03-03T19:00:00+00:00         10.6  27.489346      51
1  2017-03-03T19:20:00+00:00         11.0  28.663082      51
2  2017-03-03T20:00:00+00:00          6.3  28.307039      51
3  2017-03-03T21:55:00+00:00         10.0  29.797449      51
4  2017-03-03T23:00:00+00:00          5.4  28.307039      51


In [31]:
# CAREFUL WHEN RUNNING THIS CODE! Might rewrite data
df.to_csv('building_energy_data.csv')